In [182]:
import os
import tensorflow as tf
import pickle
from utils import LABELS, preprocess_data
import random
import numpy as np

In [183]:
def split_train_test(x, y, test_size=0.3):
    # shuffle data
    temp = list(zip(x, y))
    random.shuffle(temp)
    x, y = zip(*temp)
    x, y = list(x), list(y)
    
    # split data
    cutoff_index = int(len(x) * (1 - test_size))
    x_train = x[:cutoff_index]
    x_test = x[cutoff_index:]
    y_train = y[:cutoff_index]
    y_test = y[cutoff_index:]

    return x_train, y_train, x_test, y_test


In [184]:
data_root = os.path.abspath('raw_data')
all_paths = list(tf.io.gfile.glob(data_root + r'/*/*'))
if not all_paths:
    raise ValueError('Image dataset directory is empty.')

x = []
y = []

for path in all_paths:
    label = os.path.basename(os.path.dirname(path))
    hand_data = pickle.load(open(path, 'rb'))
    for frame in hand_data:
        processed_data = preprocess_data(frame.hand)
        x.append(processed_data)
        y.append(LABELS[label])

assert len(x) == len(y)
print(f"Total data length: {len(x)}")

_x_train, _y_train, _x_test, _y_test = split_train_test(x, y)
print(f"Train data length: {len(_x_train)}")
print(f"Test data length: {len(_x_test)}")

x_train = np.array(_x_train)
y_train = np.array(_y_train)
x_test = np.array(_x_test)
y_test = np.array(_y_test)

Total data length: 3900
Train data length: 2730
Test data length: 1170


In [185]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(21, 3)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(8)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1)

In [186]:
model.fit(x_train, y_train, epochs=28, callbacks=[callback])

Epoch 1/28
86/86 [==============================] - 0s 509us/step - loss: 1.5130 - accuracy: 0.4619
Epoch 2/28
86/86 [==============================] - 0s 498us/step - loss: 0.8394 - accuracy: 0.8143
Epoch 3/28
86/86 [==============================] - 0s 494us/step - loss: 0.4901 - accuracy: 0.9227
Epoch 4/28
86/86 [==============================] - 0s 492us/step - loss: 0.3185 - accuracy: 0.9615
Epoch 5/28
86/86 [==============================] - 0s 498us/step - loss: 0.2092 - accuracy: 0.9802
Epoch 6/28
86/86 [==============================] - 0s 502us/step - loss: 0.1595 - accuracy: 0.9886
Epoch 7/28
86/86 [==============================] - 0s 497us/step - loss: 0.1195 - accuracy: 0.9927
Epoch 8/28
86/86 [==============================] - 0s 520us/step - loss: 0.0891 - accuracy: 0.9960
Epoch 9/28
86/86 [==============================] - 0s 529us/step - loss: 0.0701 - accuracy: 0.9985
Epoch 10/28
86/86 [==============================] - 0s 527us/step - loss: 0.0584 - accuracy: 0.9985

In [187]:
model.evaluate(x_test,  y_test, verbose=2) # type: ignore

37/37 - 0s - loss: 0.0043 - accuracy: 1.0000 - 77ms/epoch - 2ms/step


[0.004328994080424309, 1.0]

In [188]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model.save('gesture_recognizer.keras')